# CLUSTERING AND COMPARING VENUES IN THE NEIGHBORHOODS OF NEW YORK CITY AND TORONTO

In [1]:
#import all libraries
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
print('Ready!')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


## NEW YORK

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
nyc_neighborhoods_data = newyork_data['features']
nyc_neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [5]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

nyc_neighborhoods = pd.DataFrame(columns=column_names)

for data in nyc_neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc_neighborhoods = nyc_neighborhoods.append({'Borough': borough,
                                                  'Neighborhood': neighborhood_name,
                                                  'Latitude': neighborhood_lat,
                                                  'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
nyc_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nyc_neighborhoods['Borough'].unique()),
        nyc_neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [8]:
nnvc = nyc_neighborhoods['Neighborhood'].value_counts()
nnvc[nnvc > 1]

Bay Terrace    2
Chelsea        2
Murray Hill    2
Sunnyside      2
Name: Neighborhood, dtype: int64

In [9]:
nyc_neighborhoods[nyc_neighborhoods['Neighborhood'] == 'Bay Terrace']


,Borough,Neighborhood,Latitude,Longitude
175,Queens,Bay Terrace,40.782843,-73.776802
235,Staten Island,Bay Terrace,40.553988,-74.139166


In [ ]:
for i in range(nyc_neighborhoods.shape[0]):
    nyn_ = nyc_neighborhoods.loc[i, 'Neighborhood']
    if nyc_neighborhoods[nyc_neighborhoods['Neighborhood'] == nyn_].shape[0] > 1:
        ind_ = nyc_neighborhoods[nyc_neighborhoods['Neighborhood'] == nyn_].index.tolist()
        for j in ind_:
            nyb__ = nyc_neighborhoods.loc[j, 'Borough']
            nyc_neighborhoods.loc[j, 'Neighborhood'] = nyn_ + ', ' + nyb__
            

In [10]:
nyc_neighborhoods[nyc_neighborhoods['Neighborhood'].str.startswith('Bay Terrace')]

,Borough,Neighborhood,Latitude,Longitude
175,Queens,Bay Terrace,40.782843,-73.776802
235,Staten Island,Bay Terrace,40.553988,-74.139166


In [11]:

address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
#Create a map of New York 
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(nyc_neighborhoods['Latitude'], nyc_neighborhoods['Longitude'], nyc_neighborhoods['Borough'], nyc_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [13]:
#Foursquare inf
CLIENT_ID = 'ZM5DFGCOMQG4A0J44L502ED32VTJIIKUNA2DEITZBLEBC5K5' 
CLIENT_SECRET = 'LUNOQVEOTST2DZBAQGWRK3OSTACWZX24WKBE2YFBBZPWQO1N' 
VERSION = '20180605'

print('ok.')

ok.


In [14]:

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):

 
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('•', end='')
            
        # create the API request from URL
        url = ('https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}'
               '&v={}&ll={},{}&intent=browse&radius={}&limit={}'
               .format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT))
            
        # make the GET request
        results = None
        while results is None:
            try:
                results = requests.get(url).json()["response"]["venues"]
            except:
                print('X', end='')
                results = None
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['name'], v['location']['lat'], 
                             v['location']['lng'], v['categories'][0]['name']) 
                            for v in results if len(v['categories']) > 0])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                             'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [15]:
nyc_venues = getNearbyVenues(names=nyc_neighborhoods['Neighborhood'],
                             latitudes=nyc_neighborhoods['Latitude'],
                             longitudes=nyc_neighborhoods['Longitude']
                            )

••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••

In [16]:
nyc_venues = nyc_venues[~nyc_venues['Venue Category'].isin(['Building', 'Office', 'Bus Line', 'Bus Station', 'Bus Stop', 'Road'])]
print(nyc_venues.shape)
nyc_venues.head()

(23883, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
1,Wakefield,40.894705,-73.847201,Pitman Deli,40.894149,-73.845748,Food
2,Wakefield,40.894705,-73.847201,Julio C Barber Shop 2,40.892648,-73.855725,Salon / Barbershop
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop


In [17]:
nyc_venues.groupby('Neighborhood').size()

Neighborhood
Allerton                      84
Annadale                      77
Arden Heights                 69
Arlington                     73
Arrochar                      77
Arverne                       80
Astoria                       73
Astoria Heights               63
Auburndale                    61
Bath Beach                    84
Battery Park City             87
Bay Ridge                     86
Bay Terrace                  159
Baychester                    80
Bayside                       85
Bayswater                     84
Bedford Park                  68
Bedford Stuyvesant            86
Beechhurst                    87
Bellaire                      79
Belle Harbor                  81
Bellerose                     82
Belmont                       62
Bensonhurst                   84
Bergen Beach                  79
Blissville                    81
Bloomfield                    78
Boerum Hill                   90
Borough Park                  35
Breezy Point                  

In [18]:
nyc_neighborhoods.shape[0], nyc_venues['Neighborhood'].unique().shape[0]

(306, 302)

In [20]:
nyc_excluded_neighborhoods = set(nyc_neighborhoods['Neighborhood']).difference(nyc_venues['Neighborhood'])
nyc_excluded_neighborhoods

set()

In [21]:
print('There are {} uniques categories.'.format(len(nyc_venues['Venue Category'].unique())))

There are 580 uniques categories.


In [22]:
#Performing one-hot on the venue category variable

nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
# we used Neighborhood_" instead of just "Neighborhood" because
# there is a venue category called "Neighborhood"
nyc_onehot['Neighborhood_'] = nyc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

nyc_onehot.head()

,Neighborhood_,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baggage Claim,Baggage Locker,Bakery,Ballroom,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Burrito Place,Business Center,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campaign Office,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Caribbean Restaurant,Carpet Store,Casino,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Football Field,College Gym,College Lab,College Library,College Math Building,College Quad,College Rec Center,College Residence Hall,College Science Building,College Stadium,College Technology Building,College Theater,College Track,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Corporate Amenity,Corporate Cafeteria,Cosmetics Shop,Costume Shop,Country Dance Club,Courthouse,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Driving School,Drugstore,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Empanada Restaurant,Entertainment Service,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fire Station,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Himalayan Restaurant,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,

In [23]:
nyc_grouped = nyc_onehot.groupby('Neighborhood_').mean().reset_index()
nyc_grouped.head()

,Neighborhood_,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baggage Claim,Baggage Locker,Bakery,Ballroom,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Burrito Place,Business Center,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campaign Office,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Caribbean Restaurant,Carpet Store,Casino,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Football Field,College Gym,College Lab,College Library,College Math Building,College Quad,College Rec Center,College Residence Hall,College Science Building,College Stadium,College Technology Building,College Theater,College Track,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Corporate Amenity,Corporate Cafeteria,Cosmetics Shop,Costume Shop,Country Dance Club,Courthouse,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Driving School,Drugstore,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Empanada Restaurant,Entertainment Service,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fire Station,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Himalayan Restaurant,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,

In [24]:
def return_most_common_venues(row, num_top_cat):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_cat]


num_top_cat = 7
indicators = ['st', 'nd', 'rd']

In [25]:
columns = ['Neighborhood_']
for ind in np.arange(num_top_cat):
    try:
        columns.append('{}{} Most Common Category'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Category'.format(ind+1))

# create a new dataframe
nyc_neighborhoods_categories_sorted = pd.DataFrame(columns=columns)
nyc_neighborhoods_categories_sorted['Neighborhood_'] = nyc_grouped['Neighborhood_']

for ind in np.arange(nyc_grouped.shape[0]):
    nyc_neighborhoods_categories_sorted.iloc[ind, 1:] = return_most_common_venues(
        nyc_grouped.iloc[ind, :], num_top_cat)

nyc_neighborhoods_categories_sorted.head()

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
0,Allerton,Salon / Barbershop,Laundry Service,Food,Non-Profit,Spa,Gas Station,Car Wash
1,Annadale,Salon / Barbershop,Pizza Place,Tattoo Parlor,Nail Salon,American Restaurant,Spa,General Entertainment
2,Arden Heights,Pool,Salon / Barbershop,Church,Moving Target,Playground,General Entertainment,Food
3,Arlington,Church,Residential Building (Apartment / Condo),Hardware Store,Professional & Other Places,Deli / Bodega,Salon / Barbershop,Scenic Lookout
4,Arrochar,Deli / Bodega,Food Truck,Pizza Place,Nail Salon,Italian Restaurant,Dry Cleaner,Liquor Store


In [26]:
# set number of clusters
kclusters = 5

nyc_grouped_clustering = nyc_grouped.drop('Neighborhood_', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 2, 2, 2, 2, 4, 2, 1, 2], dtype=int32)

In [27]:
nyc_neighborhoods_categories_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

nyc_merged = nyc_neighborhoods.rename(columns={'Neighborhood': 'Neighborhood_'}).copy()
nyc_merged = nyc_merged[~nyc_merged['Neighborhood_'].isin(nyc_excluded_neighborhoods)]

nyc_merged = nyc_merged.join(nyc_neighborhoods_categories_sorted.set_index('Neighborhood_'), on='Neighborhood_')

nyc_merged.head()

,Borough,Neighborhood_,Latitude,Longitude,Cluster Labels,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
0,Bronx,Wakefield,40.894705,-73.847201,3,Salon / Barbershop,Laundry Service,Church,Music Venue,Food,Gas Station,Playground
1,Bronx,Co-op City,40.874294,-73.829939,4,School,Residential Building (Apartment / Condo),Dentist's Office,Financial or Legal Service,Church,Other Great Outdoors,Hardware Store
2,Bronx,Eastchester,40.887556,-73.827806,1,Automotive Shop,Auto Dealership,Caribbean Restaurant,Gas Station,Metro Station,Fast Food Restaurant,Bridge
3,Bronx,Fieldston,40.895437,-73.905643,2,Synagogue,College Residence Hall,Residential Building (Apartment / Condo),College Administrative Building,Student Center,Church,School
4,Bronx,Riverdale,40.890834,-73.912585,4,Residential Building (Apartment / Condo),Synagogue,Doctor's Office,Park,Playground,Athletics & Sports,General College & University


In [28]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Latitude'], nyc_merged['Longitude'], nyc_merged['Neighborhood_'], nyc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
#CLUSTER 1
nyc_merged.loc[nyc_merged['Cluster Labels'] == 0, 
               nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
8,Norwood,Doctor's Office,Park,Deli / Bodega,Residential Building (Apartment / Condo),Medical Center,Laundry Service,Food Truck
11,Pelham Parkway,Doctor's Office,Dentist's Office,Deli / Bodega,Residential Building (Apartment / Condo),Diner,Nail Salon,Pizza Place
33,Morris Park,Doctor's Office,Laundry Service,Pizza Place,Residential Building (Apartment / Condo),Bank,Salon / Barbershop,Deli / Bodega
38,Schuylerville,Doctor's Office,Salon / Barbershop,Deli / Bodega,Pharmacy,Professional & Other Places,Pizza Place,Mental Health Office
42,Pelham Gardens,Doctor's Office,Other Great Outdoors,Pharmacy,Pizza Place,Playground,Medical Center,Donut Shop
53,Manhattan Terrace,Doctor's Office,Residential Building (Apartment / Condo),Dentist's Office,Housing Development,Parking,Medical Center,School
64,Brooklyn Heights,Doctor's Office,Residential Building (Apartment / Condo),Church,Salon / Barbershop,School,Laundry Service,Women's Store
73,Starrett City,Doctor's Office,Laundry Service,Residential Building (Apartment / Condo),Parking,Pharmacy,Middle School,Gym / Fitness Center
81,Dyker Heights,Doctor's Office,Gas Station,Salon / Barbershop,Deli / Bodega,Automotive Shop,Event Space,Dog Run
99,Fort Hamilton,Doctor's Office,Pizza Place,Residential Building (Apartment / Condo),Deli / Bodega,Laundry Service,Salon / Barbershop,Italian Restaurant


In [32]:
# CLUSTER 2
nyc_merged.loc[nyc_merged['Cluster Labels'] == 1, 
               nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
2,Eastchester,Automotive Shop,Auto Dealership,Caribbean Restaurant,Gas Station,Metro Station,Fast Food Restaurant,Bridge
23,Longwood,Pizza Place,Automotive Shop,Train,Mexican Restaurant,Church,Salon / Barbershop,Grocery Store
24,Hunts Point,Automotive Shop,Factory,Food,School,Pizza Place,Gas Station,Bank
148,South Ozone Park,Gas Station,Government Building,Deli / Bodega,Taxi,Moving Target,Bar,Park
149,College Point,Automotive Shop,Salon / Barbershop,Bank,Mexican Restaurant,Deli / Bodega,Sandwich Place,Gas Station
152,Auburndale,Automotive Shop,Train,Nail Salon,Deli / Bodega,Event Space,Laundry Service,Lawyer
164,South Jamaica,Automotive Shop,Church,Laundry Service,Salon / Barbershop,Pizza Place,Deli / Bodega,Caribbean Restaurant
167,Springfield Gardens,Church,Automotive Shop,High School,Gas Station,Chinese Restaurant,Check Cashing Service,Fried Chicken Joint
173,Steinway,Automotive Shop,Gym / Fitness Center,Deli / Bodega,Baseball Field,Bank,Pizza Place,Miscellaneous Shop
192,Somerville,Automotive Shop,Harbor / Marina,Bank,Deli / Bodega,Housing Development,Plane,Pizza Place


In [33]:
#CLUSTER 3
nyc_merged.loc[nyc_merged['Cluster Labels'] == 2, 
               nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
3,Fieldston,Synagogue,College Residence Hall,Residential Building (Apartment / Condo),College Administrative Building,Student Center,Church,School
5,Kingsbridge,Laundry Service,Salon / Barbershop,Miscellaneous Shop,Boutique,Bank,Cosmetics Shop,Thrift / Vintage Store
7,Woodlawn,Bar,Salon / Barbershop,Deli / Bodega,Pizza Place,Pub,Food & Drink Shop,Gas Station
10,Baychester,Gas Station,Bank,Donut Shop,Fast Food Restaurant,Sandwich Place,Rental Car Location,Electronics Store
12,City Island,Harbor / Marina,Italian Restaurant,Miscellaneous Shop,Park,Thrift / Vintage Store,Beach,Grocery Store
14,University Heights,College Academic Building,Deli / Bodega,Salon / Barbershop,Supermarket,Bar,Pizza Place,College Gym
16,Fordham,Clothing Store,Doctor's Office,Deli / Bodega,Women's Store,Nail Salon,Salon / Barbershop,Sporting Goods Shop
18,West Farms,Automotive Shop,Residential Building (Apartment / Condo),School,Doctor's Office,Moving Target,Deli / Bodega,Chinese Restaurant
22,Port Morris,Factory,Government Building,General Entertainment,Residential Building (Apartment / Condo),Storage Facility,Distillery,Gas Station
25,Morrisania,Church,High School,Chinese Restaurant,Housing Development,Residential Building (Apartment / Condo),Deli / Bodega,Laundry Service


In [34]:
#CLUSTER 4
nyc_merged.loc[nyc_merged['Cluster Labels'] == 3, 
               nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
0,Wakefield,Salon / Barbershop,Laundry Service,Church,Music Venue,Food,Gas Station,Playground
6,Marble Hill,Deli / Bodega,Salon / Barbershop,High School,Laundry Service,Residential Building (Apartment / Condo),Church,Furniture / Home Store
9,Williamsbridge,Church,Salon / Barbershop,Caribbean Restaurant,Hardware Store,Deli / Bodega,School,Spa
17,East Tremont,Salon / Barbershop,Food,Cosmetics Shop,Church,Shoe Store,Event Space,Jewelry Store
19,High Bridge,Salon / Barbershop,Deli / Bodega,Pharmacy,Laundry Service,Residential Building (Apartment / Condo),Chinese Restaurant,Pizza Place
32,Van Nest,Deli / Bodega,Salon / Barbershop,Laundry Service,Pizza Place,Nail Salon,Residential Building (Apartment / Condo),Diner
34,Belmont,Salon / Barbershop,Italian Restaurant,Deli / Bodega,School,Park,Laundry Service,Dessert Shop
41,Olinville,Salon / Barbershop,Deli / Bodega,Caribbean Restaurant,Food,Chinese Restaurant,Health & Beauty Service,Bakery
43,Concourse,Salon / Barbershop,Residential Building (Apartment / Condo),Pizza Place,Deli / Bodega,Nail Salon,Spanish Restaurant,Financial or Legal Service
44,Unionport,Salon / Barbershop,Laundry Service,Deli / Bodega,Shoe Store,Grocery Store,Spanish Restaurant,Post Office


In [35]:
#CLUSTER 5
nyc_merged.loc[nyc_merged['Cluster Labels'] == 4, 
               nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood_,1st Most Common Category,2nd Most Common Category,3rd Most Common Category,4th Most Common Category,5th Most Common Category,6th Most Common Category,7th Most Common Category
1,Co-op City,School,Residential Building (Apartment / Condo),Dentist's Office,Financial or Legal Service,Church,Other Great Outdoors,Hardware Store
4,Riverdale,Residential Building (Apartment / Condo),Synagogue,Doctor's Office,Park,Playground,Athletics & Sports,General College & University
13,Bedford Park,Residential Building (Apartment / Condo),Deli / Bodega,Church,Salon / Barbershop,Doctor's Office,Park,Laundry Service
15,Morris Heights,Residential Building (Apartment / Condo),Grocery Store,Convenience Store,School,Salon / Barbershop,Deli / Bodega,Spanish Restaurant
20,Melrose,Residential Building (Apartment / Condo),Doctor's Office,Salon / Barbershop,Laundry Service,High School,Housing Development,Deli / Bodega
21,Mott Haven,Residential Building (Apartment / Condo),Salon / Barbershop,Grocery Store,Chinese Restaurant,Church,Restaurant,Pharmacy
26,Soundview,Residential Building (Apartment / Condo),Housing Development,Laundry Service,Deli / Bodega,Salon / Barbershop,Chinese Restaurant,Pizza Place
30,Parkchester,Salon / Barbershop,Residential Building (Apartment / Condo),Laundry Service,Convenience Store,Indian Restaurant,School,Doctor's Office
35,Spuyten Duyvil,Residential Building (Apartment / Condo),Doctor's Office,Medical Center,Salon / Barbershop,Dentist's Office,Thai Restaurant,American Restaurant
45,Edenwald,Residential Building (Apartment / Condo),Salon / Barbershop,Church,Liquor Store,Deli / Bodega,Gas Station,Laundry Service


## TORONTO 

In [36]:

tor_neighborhoodss = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
tor_neighborhoods = tor_neighborhoodss[0]

ImportError: lxml not found, please install it

In [ ]:

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(tor_neighborhoods['Borough'].unique()),
        tor_neighborhoods.shape[0]
    )
)

In [ ]:
tor_neighborhoods.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [ ]:
#Removing records where the borough is "not assigned"
tor_neighborhoods[tor_neighborhoods.Borough == 'Not assigned']['Neighborhood'].unique()

In [ ]:
print("There are {} records where the borough is not assigned".format(
    tor_neighborhoods[tor_neighborhoods.Borough == 'Not assigned'].shape[0]))

In [ ]:
tor_neighborhoods = tor_neighborhoods[tor_neighborhoods.Borough != 'Not assigned']

In [ ]:
tor_neighborhoods[tor_neighborhoods.PostalCode == 'M5A']

In [ ]:
tor_neighborhoods = (tor_neighborhoods.groupby(['PostalCode', 'Borough'])['Neighborhood']
      .apply(lambda x: "{}".format(', '.join(x))).reset_index())
tor_neighborhoods[tor_neighborhoods.Neighborhood.str.contains('Not assigned')]

In [ ]:
tor_neighborhoods.loc[tor_neighborhoods.Neighborhood.str.contains('Not assigned'), 'Neighborhood'] = \
    tor_neighborhoods.loc[tor_neighborhoods.Neighborhood.str.contains('Not assigned'), 'Borough']

In [ ]:
tor_neighborhoods[tor_neighborhoods.Borough == "Queen's Park"]

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_1fdadd8e87cf46c0b09f29d2f94d1677 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GqQatGmNFhg-kutc04UdjsXbQ7C_XEhQLOfeOqSsA_8q',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_1fdadd8e87cf46c0b09f29d2f94d1677.get_object(Bucket='finalprojectthebattleofneigborhoo-donotdelete-pr-wgiugipff8rsm0',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

tor_lat_lng_df = pd.read_csv(body)
tor_lat_lng_df.head()

In [ ]:
#Adding langitude and latitude to the neighborhoods dataframe
tor_neighborhoods.reset_index(drop=True, inplace=True)
tor_neighborhoods['Latitude'] = -99999.9
tor_neighborhoods['Longitude'] = -99999.9

for i in range(tor_neighborhoods.shape[0]):
    postalcode = tor_neighborhoods.loc[i, 'PostalCode']
    lat = tor_lat_lng_df.loc[tor_lat_lng_df['Postal Code'] == postalcode, 'Latitude'].squeeze()
    lng = tor_lat_lng_df.loc[tor_lat_lng_df['Postal Code'] == postalcode, 'Longitude'].squeeze()
    tor_neighborhoods.loc[i, 'Latitude'] = lat
    tor_neighborhoods.loc[i, 'Longitude'] = lng

tor_neighborhoods.head()

In [ ]:
tor_neighborhoods.shape

In [ ]:
#Using geopy Library to get the Coordinates of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
#Creating a map of Toronto with neighborhoods superimposed on top
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10,
                         min_zoom=9, max_zoom=11)

# add neighborhood markers to map
for lat, lng, borough, neighborhood in zip(tor_neighborhoods['Latitude'], tor_neighborhoods['Longitude'], 
                                           tor_neighborhoods['Borough'], tor_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        weight=2,
        popup=label,
        color='#333333',
        fill=True,
        fill_color='#388e3c',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
tor_venues = getNearbyVenues(names=tor_neighborhoods['Neighborhood'],
                             latitudes=tor_neighborhoods['Latitude'],
                             longitudes=tor_neighborhoods['Longitude']
                            )

In [ ]:
# Removing records where venue is "building" or "office"
tor_venues = tor_venues[~tor_venues['Venue Category'].isin(['Building', 'Office', 'Bus Line', 'Bus Station', 'Bus Stop', 'Road'])]
print(tor_venues.shape)
tor_venues.head()

In [ ]:
#Let's check how many venues were returned for each neighborhoo
tor_venues.groupby('Neighborhood').size()

In [ ]:
tor_neighborhoods.shape[0], tor_venues['Neighborhood'].unique().shape[0]

In [ ]:
tor_excluded_neighborhoods = set(tor_neighborhoods['Neighborhood']).difference(set(tor_venues['Neighborhood']))
tor_excluded_neighborhoods

In [ ]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
# we used Neighborhood_" instead of just "Neighborhood" because
# there is a venue category called "Neighborhood"
tor_onehot['Neighborhood_'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()


In [ ]:
tor_grouped = tor_onehot.groupby('Neighborhood_').mean().reset_index()
tor_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_cat):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_cat]


num_top_cat = 7
indicators = ['st', 'nd', 'rd']


In [ ]:
columns = ['Neighborhood_']
for ind in np.arange(num_top_cat):
    try:
        columns.append('{}{} Most Common Category'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Category'.format(ind+1))

# create a new dataframe
tor_neighborhoods_categories_sorted = pd.DataFrame(columns=columns)
tor_neighborhoods_categories_sorted['Neighborhood_'] = tor_grouped['Neighborhood_']

for ind in np.arange(tor_grouped.shape[0]):
    tor_neighborhoods_categories_sorted.iloc[ind, 1:] = return_most_common_venues(
        tor_grouped.iloc[ind, :], num_top_cat)

tor_neighborhoods_categories_sorted.head()

In [ ]:
#CLUSTERING 

In [ ]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood_', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
tor_neighborhoods_categories_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_neighborhoods.rename(columns={'Neighborhood': 'Neighborhood_'}).copy()
tor_merged = tor_merged[~tor_merged['Neighborhood_'].isin(tor_excluded_neighborhoods)]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(tor_neighborhoods_categories_sorted.set_index('Neighborhood_'), on='Neighborhood_')

tor_merged.head()

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood_'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [ ]:
#CLUSTER 1 
tor_merged.loc[tor_merged['Cluster Labels'] == 0, 
               tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]


In [ ]:
#CLUSTER 2
tor_merged.loc[tor_merged['Cluster Labels'] == 1, 
               tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

In [ ]:
#CLUSTER 3
tor_merged.loc[tor_merged['Cluster Labels'] == 2, 
               tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

In [ ]:
#CLUSTER 4
tor_merged.loc[tor_merged['Cluster Labels'] == 3, 
               tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

In [ ]:
#CLUSTER 5
tor_merged.loc[tor_merged['Cluster Labels'] == 4, 
               tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

In [ ]:
# READY TO PLOT 

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import folium # map rendering library
from IPython.display import HTML
import time

In [ ]:
%config InlineBackend.figure_format = 'retina'
plt.rc('figure', dpi=300)
plt.rc('savefig', dpi=300)
fig_size = (12,6)
big_fig_size = (18,8)
fig_fc = '#ffffff'
pc = ["#4285f4", "#db4437", "#f4b400", "#0f9d58", "#ab47bc", "#00acc1", "#ff7043", 
      "#9e9d24", "#5c6bc0", "#f06292", "#00796b", "#c2185b", "#7e57c2", "#03a9f4", 
      "#8bc34a", "#fdd835", "#fb8c00", "#8d6e63", "#9e9e9e", "#607d8b"]

def plot_conf(ax, xlbl='', ylbl='', t=''):
    """
    This function perform operations to produce better-looking 
    visualizations
    """
    # changing the background color of the plot
    ax.set_facecolor('#ffffff')
    # modifying the ticks on plot axes
    ax.tick_params(axis='both', labelcolor='#616161', color='#ffffff')
    ax.tick_params(axis='both', which='major', labelsize=9)
    # adding a grid and specifying its color
    ax.grid(True, color='#e9e9e9')
    # making the grid appear behind the graph elements
    ax.set_axisbelow(True)
    # hiding axes
    ax.spines['bottom'].set_color('#ffffff')
    ax.spines['top'].set_color('#ffffff') 
    ax.spines['right'].set_color('#ffffff')
    ax.spines['left'].set_color('#ffffff')
    # setting the title, x label, and y label of the plot
    ax.set_title(t, fontsize=14, color='#616161', loc='left', pad=24, fontweight='bold');
    ax.set_xlabel(xlbl, labelpad=16, fontsize=11, color='#616161', fontstyle='italic');
    ax.set_ylabel(ylbl, color='#616161', labelpad=16, fontsize=11, fontstyle='italic');
    
# table configuration
styles = [
    dict(selector="td, th", props=[("border", "1px solid #333"), ("padding", "2px")]),
    dict(selector="th.col_heading", props=[("background", "#eee8d5"), ("color", "#b58900"), ("padding", "5px 8px")]),
    dict(selector="th.index_name", props=[("background", "#eee8d5"), ("color", "#268bd2"), ("padding", "5px 8px")]),
    dict(selector="th.blank", props=[("background", "#eee8d5"), ("color", "#268bd2"), ("padding", "0")]),
    dict(selector="th.row_heading.level0", props=[("background", "rgba(133, 153, 0, 0.1)")]),
    dict(selector="th.row_heading.level1", props=[("background", "rgba(42, 161, 152, 0.1)")]),
    dict(selector="thead tr:nth-child(2) th", props=[("border-bottom", "3px solid #333333")]),
    dict(selector="td:hover", props=[("font-weight", "bold"), ("background", "#002b36"), ("color", "Gold")]),
]

disp_fmt = "<h2 style='color: #b58900'>{}<span style='color: #FF91CE; font-size: 115%'>:</span></h2>"

In [ ]:
#Most Common Venue Categories in NYC

fig, ax = plt.subplots(figsize=fig_size, facecolor=fig_fc)
nyc_top_vc = nyc_venues['Venue Category'].value_counts(normalize=False)
ax = nyc_top_vc.head(15).plot(kind='barh', color=pc[0]);
ax.invert_yaxis()
plot_conf(ax, xlbl='Number of venues', ylbl='Venue category', t='')
plt.title("Most Common Venue Categories in NYC")
plt.tight_layout()

In [ ]:
#Most Common Venue Categories in NYC
fig, ax = plt.subplots(figsize=fig_size, facecolor=fig_fc)
tor_top_vc = tor_venues['Venue Category'].value_counts(normalize=False)
ax = tor_top_vc.head(15).plot(kind='barh', color=pc[3]);
ax.invert_yaxis()
plot_conf(ax, xlbl='Number of venues', ylbl='Venue category', t='')
plt.tight_layout()
plt.title("Most Common Venue Categories in Toronto")

In [ ]:
# Categories in NYC Neighborhoods
fig, ax = plt.subplots(figsize=fig_size, facecolor=fig_fc)
nyc_g = nyc_onehot.groupby('Neighborhood_').max().reset_index()
nyc_p = nyc_g.drop('Neighborhood_', axis=1).sum().sort_values(ascending=False)
print("{} neighborhoods in NYC".format(nyc_g.shape[0]))
ax = nyc_p.head(15).plot(kind='barh', color=pc[8]);
ax.invert_yaxis()
plot_conf(ax, xlbl='Number of neighborhoods', ylbl='Venue category', t='')
plt.tight_layout()
plt.title("Venue Categories Exist in More NYC Neighborhoods")

In [ ]:
# Categories in TO Neighborhoods
fig, ax = plt.subplots(figsize=fig_size, facecolor=fig_fc)
tor_g = tor_onehot.groupby('Neighborhood_').max().reset_index()
tor_p = tor_g.drop('Neighborhood_', axis=1).sum().sort_values(ascending=False)
print("{} neighborhoods in Toronto".format(tor_g.shape[0]))
ax = tor_p.head(15).plot(kind='barh', color=pc[14]);
ax.invert_yaxis()
plot_conf(ax, xlbl='Number of neighborhoods', ylbl='Venue category', t='')
plt.tight_layout()
plt.title("Venue Categories Exist in More Toronto Neighborhoods")

In [ ]:
# Common Venue Categories AT NYC and Toronto


In [ ]:
clmns_nyc_only = set(nyc_grouped.columns).difference(set(tor_grouped.columns))
clmns_tor_only = set(tor_grouped.columns).difference(set(nyc_grouped.columns))

In [ ]:
nyc_grouped_ = nyc_grouped.copy()
nyc_grouped_['Neighborhood_'] = nyc_grouped_['Neighborhood_'].apply(lambda x: x + '_NYC')
tor_grouped_ = tor_grouped.copy()
tor_grouped_['Neighborhood_'] = tor_grouped_['Neighborhood_'].apply(lambda x: x + '_Toronto')

for c in clmns_tor_only:
    nyc_grouped_[c] = 0
for c in clmns_nyc_only:
    tor_grouped_[c] = 0
    
all_clmns_sorted = ['Neighborhood_'] + sorted(list(nyc_grouped_.drop('Neighborhood_', axis=1).columns), key=str.lower)
nyc_grouped_ = nyc_grouped_[all_clmns_sorted]
tor_grouped_ = tor_grouped_[all_clmns_sorted]
    
nyc_tor_grouped = pd.concat([nyc_grouped_, tor_grouped_]).reset_index(drop=True)

In [ ]:

nyc_tor_grouped.shape

In [ ]:
# obtaining the most popular categories for each neighborhood
nyc_tor_neighborhoods_categories_sorted = pd.DataFrame(columns=columns)
nyc_tor_neighborhoods_categories_sorted['Neighborhood_'] = nyc_tor_grouped['Neighborhood_']

for ind in np.arange(nyc_tor_grouped.shape[0]):
    nyc_tor_neighborhoods_categories_sorted.iloc[ind, 1:] = return_most_common_venues(
        nyc_tor_grouped.iloc[ind, :], num_top_cat)

nyc_tor_neighborhoods_categories_sorted.head()

In [ ]:
# ANOTHER PLOT
fig, ax = plt.subplots(figsize=fig_size, facecolor=fig_fc)
nyc_tor_top_vc = pd.concat([nyc_venues, tor_venues])['Venue Category'].value_counts(normalize=True) * 100
ax = nyc_tor_top_vc.head(15).plot(kind='barh', color=pc[12]);
plot_conf(ax, xlbl='Number of venues', ylbl='Venue category', t='')
plt.title("Most Common Venue Categories in both NYC and Toronto")